In [ ]:
import os, sys, xml.etree.ElementTree as ET, pandas as pd, csv

In [ ]:
def parse_xml(tree):
    root = tree.getroot()
    sukunimi = root.find("SukuNimi").text
    etunimi = root.find("KutsumaNimi").text
    kokonimi = (f'{etunimi}_{sukunimi}').replace(" ", "") #some entries contain whitespace

    syntymapaiva = root.find("SyntymaPvm").text
    sukupuoli = root.find("SukuPuoliKoodi").text

    toimielimet = root.findall(".//Toimielin")
    toimielinjasenyydet = []

    # kerätään puhemiehistön jäsenet ja valiokuntien pj:t ja vpj:t
    for toimielin in toimielimet:
        if toimielin.find('Nimi').text == 'Puhemiehistö':
            paikka = toimielin.find('Nimi').text
            rooli = toimielin.find('Jasenyys/Rooli').text
            alku = toimielin.find('Jasenyys/AlkuPvm').text
            loppu = toimielin.find('Jasenyys/LoppuPvm').text
            toimielinjasenyydet.append([paikka, rooli, alku, loppu])
        if toimielin.attrib.get('OnkoValiokunta') == 'true':
            paikka = toimielin.find('Nimi').text
            if toimielin.find('Jasenyys/Rooli').text == 'Puheenjohtaja' or toimielin.find('Jasenyys/Rooli').text == 'Varapuheenjohtaja':
                rooli = toimielin.find('Jasenyys/Rooli').text
                alku = toimielin.find('Jasenyys/AlkuPvm').text
                loppu = toimielin.find('Jasenyys/LoppuPvm').text
                toimielinjasenyydet.append([paikka, rooli, alku, loppu])

    data[kokonimi] = [syntymapaiva, sukupuoli, toimielinjasenyydet]

In [ ]:
files = os.listdir('mop')
data = {}

for file in files:
    tree = ET.parse(f'mop/{file}')
    parse_xml(tree)

filename = 'parliamentary_roles.csv'
(pd.DataFrame.from_dict(data=data, orient='index').to_csv(filename, header=False))